# Customer churn prediction using Keras

Set current working directory

In [1]:
import os

current_directory = "/media/satya/backupdata/Udemy-Deep-Learning-A-Z/Deep Learning A-Z/Volume 1 - Supervised Deep Learning/Part 1 - Artificial Neural Networks (ANN)/Artificial_Neural_Networks"
os.chdir(current_directory)

Importing required libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Importing the dataset

In [3]:
dataset = pd.read_csv("Churn_Modelling.csv")
dataset.head()

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         79084.10       0

In [4]:
# Get the datatype of columns.
dataset.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
# See statistics of numeric columns
dataset.describe()

RowNumber    CustomerId   CreditScore           Age        Tenure  \
count  10000.00000  1.000000e+04  10000.000000  10000.000000  10000.000000   
mean    5000.50000  1.569094e+07    650.528800     38.921800      5.012800   
std     2886.89568  7.193619e+04     96.653299     10.487806      2.892174   
min        1.00000  1.556570e+07    350.000000     18.000000      0.000000   
25%     2500.75000  1.562853e+07    584.000000     32.000000      3.000000   
50%     5000.50000  1.569074e+07    652.000000     37.000000      5.000000   
75%     7500.25000  1.575323e+07    718.000000     44.000000      7.000000   
max    10000.00000  1.581569e+07    850.000000     92.000000     10.000000   

             Balance  NumOfProducts    HasCrCard  IsActiveMember  \
count   10000.000000   10000.000000  10000.00000    10000.000000   
mean    76485.889288       1.530200      0.70550        0.515100   
std     62397.405202       0.581654      0.45584        0.499797   
min         0.000000       1.000000      0.00000        0.000000   
25%         0.000000       1.000000      0.00000        0.000000   
50%     97198.540000       1.000000      1.00000        1.000000   
75%    127644.240000       2.000000      1.00000        1.000000   
max    250898.090000       4.000000      1.00000        1.000000   

       EstimatedSalary        Exited  
count     10000.000000  10000.000000  
mean     100090.239881      0.203700  
std       57510.492818      0.402769  
min          11.580000      0.000000  
25%       51002.110000      0.000000  
50%      100193.915000      0.000000  
75%      149388.247500      0.000000  
max      199992.480000      1.000000

In [6]:
# See statistics of categorical columns.
dataset.Geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [7]:
dataset.Gender.value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [8]:
# Separate independent variables and dependent variable "Exited"
X = dataset.iloc[:,3:13].values
Y = dataset.iloc[:,13].values

In [9]:
type(X)

numpy.ndarray

In [10]:
# Inspect a row of X. Here row number 1 returned.
X[10,:]

array([528, 'France', 'Male', 31, 6, 102016.72, 2, 0, 0, 80181.12], dtype=object)

In [11]:
# Here we can see that columns 4 and 5 (columns Geography & Gender) are of type object.i.e. categorical.
# So, Lets encode categorical data

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Label encode column 1 of X i.e. Geography column. It will convert 3 countries into 1,2 or 3.
labelencoder_X_1 = LabelEncoder()
X[:,1] = labelencoder_X_1.fit_transform(X[:,1])

In [13]:
X[:,1]

array([0, 2, 0, ..., 0, 1, 0], dtype=object)

In [14]:
# Label encode column 2 of X i.e. Gender column. It will give 0/1 for male/female.
labelencoder_X_2 = LabelEncoder()
X[:,2] = labelencoder_X_2.fit_transform(X[:,2])

In [15]:
X[:,2]

array([0, 0, 0, ..., 0, 1, 0], dtype=object)

In [16]:
# Do one hot encoding for 0,1,2 values
onehotencoder = OneHotEncoder(categorical_features=[1])
X = onehotencoder.fit_transform(X).toarray()

# Drop one of 3 one-hot-encoded variables.
X = X[:, 1:]

In [17]:
X

array([[  0.00000000e+00,   0.00000000e+00,   6.19000000e+02, ...,
          1.00000000e+00,   1.00000000e+00,   1.01348880e+05],
       [  0.00000000e+00,   1.00000000e+00,   6.08000000e+02, ...,
          0.00000000e+00,   1.00000000e+00,   1.12542580e+05],
       [  0.00000000e+00,   0.00000000e+00,   5.02000000e+02, ...,
          1.00000000e+00,   0.00000000e+00,   1.13931570e+05],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   7.09000000e+02, ...,
          0.00000000e+00,   1.00000000e+00,   4.20855800e+04],
       [  1.00000000e+00,   0.00000000e+00,   7.72000000e+02, ...,
          1.00000000e+00,   0.00000000e+00,   9.28885200e+04],
       [  0.00000000e+00,   0.00000000e+00,   7.92000000e+02, ...,
          1.00000000e+00,   0.00000000e+00,   3.81907800e+04]])

# -----------------------------------------------------------------------------------------------------------

In [18]:
# Split dataset into train and testset

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [20]:
# Feature scaling

In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

# --------------------------------------------------------------------------------------------------------

# The ANN using Keras

In [22]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [23]:
# initializing the ANN
# https://keras.io/models/sequential/
classifier = Sequential()

In [24]:
# Adding the input layer and first hidden layer

# 11 node in input layer and 1 node in output layer. so, no_of_hidden_layer = (11 + 1)/2 = 6
# So, no of hidden units will be 6 which is = units
# https://keras.io/layers/core/#dense
# https://keras.io/initializers/#randomuniform
# https://keras.io/activations/ 

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_shape=(11,)))
classifier.add(Dropout(p = 0.1))

/home/satya/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`


In [25]:
# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(p = 0.1))

/home/satya/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  app.launch_new_instance()


In [26]:
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))


In [27]:
# Compiling the ANN
# https://keras.io/optimizers/#adam
# https://keras.io/losses/#binary_crossentropy
# https://keras.io/metrics/
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [28]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 5s - loss: 0.4852 - acc: 0.7953     
Epoch 2/100
8000/8000 [==============================] - 4s - loss: 0.4276 - acc: 0.7960     
Epoch 3/100
8000/8000 [==============================] - 4s - loss: 0.4188 - acc: 0.7960     
Epoch 4/100
8000/8000 [==============================] - 4s - loss: 0.4131 - acc: 0.8144     
Epoch 5/100
8000/8000 [==============================] - 4s - loss: 0.4155 - acc: 0.8231     
Epoch 6/100
8000/8000 [==============================] - 6s - loss: 0.4106 - acc: 0.8219     
Epoch 7/100
8000/8000 [==============================] - 5s - loss: 0.4058 - acc: 0.8219     
Epoch 8/100
8000/8000 [==============================] - 5s - loss: 0.4056 - acc: 0.8251     
Epoch 9/100
8000/8000 [==============================] - 4s - loss: 0.4050 - acc: 0.8231     
Epoch 10/100
8000/8000 [==============================] - 5s - loss: 0.4014 - acc: 0.8273     
Epoch 11/100
8000/8000 [==============================] - 4

8000/8000 [==============================] - 3s - loss: 0.3999 - acc: 0.8399     
Epoch 88/100
8000/8000 [==============================] - 3s - loss: 0.3956 - acc: 0.8419     
Epoch 89/100
8000/8000 [==============================] - 3s - loss: 0.3980 - acc: 0.8364     
Epoch 90/100
8000/8000 [==============================] - 3s - loss: 0.3995 - acc: 0.8373     
Epoch 91/100
8000/8000 [==============================] - 3s - loss: 0.3970 - acc: 0.8389     
Epoch 92/100
8000/8000 [==============================] - 3s - loss: 0.3999 - acc: 0.8356     
Epoch 93/100
8000/8000 [==============================] - 3s - loss: 0.3942 - acc: 0.8401     
Epoch 94/100
8000/8000 [==============================] - 3s - loss: 0.4003 - acc: 0.8343     
Epoch 95/100
8000/8000 [==============================] - 3s - loss: 0.3959 - acc: 0.8413     
Epoch 96/100
8000/8000 [==============================] - 3s - loss: 0.3997 - acc: 0.8363     
Epoch 97/100
8000/8000 [==============================] - 3s - 

# ---------------------------------------------------------------------------------------------------------

# Make prediction and evaluate model

In [29]:
# predicting test set results
y_pred = classifier.predict(X_test)
# y_pred is the probability of customers are going to leave the company

In [30]:
# Convert probabilities to 0 if y_pred value is <= 0.5 else convert to 1
y_pred = (y_pred > 0.5)

In [31]:
# Making the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [32]:
cm

array([[1552,   43],
       [ 250,  155]])

In [33]:
from __future__ import division
accuracy = (cm[0,0] + cm[1,1]) / (cm[0,1] + cm[1,0] + cm[0,0] + cm[1,1])
#accuracy = (1545+145)/(260+50+1545+145)

In [34]:
accuracy

0.85350000000000004

# ----------------------------------------------------------------------------------------------------------

# Predict on a new data point

In [35]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Credit Score: 600
Geography: France
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""

'Predict if the customer with the following informations will leave the bank:\nCredit Score: 600\nGeography: France\nGender: Male\nAge: 40\nTenure: 3\nBalance: 60000\nNumber of Products: 2\nHas Credit Card: Yes\nIs Active Member: Yes\nEstimated Salary: 50000'

In [36]:
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

In [37]:
new_prediction = (new_prediction > 0.5)

In [38]:
new_prediction

array([[False]], dtype=bool)

# ---------------------------------------------------------------------------------------------------------

# Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [39]:
#from keras.wrappers.scikit_learn import KerasClassifier
#from sklearn.model_selection import cross_val_score
#from keras.models import Sequential
#from keras.layers import Dense
#def build_classifier():
#    classifier = Sequential()
#    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
#    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
#    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
#    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#    return classifier

In [40]:
#classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)

In [41]:
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
#accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

#mean = accuracies.mean()

#variance = accuracies.std()

In [42]:
#if 'session' in locals() and session is not None:
#    print('Close interactive session')
#    session.close()

# -----------------------------------------------------------------------------------------------------------

# Tuning the ANN

In [43]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/100
7200/7200 [==============================] - 1s - loss: 0.6029 - acc: 0.7951     
Epoch 2/100
7200/7200 [==============================] - 0s - loss: 0.4448 - acc: 0.7974     
Epoch 3/100
7200/7200 [==============================] - 0s - loss: 0.4260 - acc: 0.8169     
Epoch 4/100
7200/7200 [==============================] - 0s - loss: 0.4212 - acc: 0.8224     
Epoch 5/100
7200/7200 [==============================] - 0s - loss: 0.4174 - acc: 0.8283     
Epoch 6/100
7200/7200 [==============================] - 1s - loss: 0.4141 - acc: 0.8292     
Epoch 7/100
7200/7200 [==============================] - 1s - loss: 0.4113 - acc: 0.8311     
Epoch 8/100
7200/7200 [==============================] - 1s - loss: 0.4093 - acc: 0.8326     
Epoch 9/100
7200/7200 [==============================] - 1s - loss: 0.4081 - acc: 0.8324     
Epoch 10/100
7200/7200 [==============================] - 1s - loss: 0.4068 - acc: 0.8336     
Epoch 11/100
7200/7200 [==============================] - 1

7200/7200 [==============================] - 1s - loss: 0.3457 - acc: 0.8571     
Epoch 88/100
7200/7200 [==============================] - 1s - loss: 0.3446 - acc: 0.8567     
Epoch 89/100
7200/7200 [==============================] - 1s - loss: 0.3435 - acc: 0.8592     
Epoch 90/100
7200/7200 [==============================] - 1s - loss: 0.3421 - acc: 0.8581     
Epoch 91/100
7200/7200 [==============================] - 1s - loss: 0.3408 - acc: 0.8610     
Epoch 92/100
7200/7200 [==============================] - 1s - loss: 0.3408 - acc: 0.8601     
Epoch 93/100
7200/7200 [==============================] - 1s - loss: 0.3395 - acc: 0.8622     
Epoch 94/100
7200/7200 [==============================] - 1s - loss: 0.3388 - acc: 0.8619     
Epoch 95/100
7200/7200 [==============================] - 1s - loss: 0.3383 - acc: 0.8626     
Epoch 96/100
7200/7200 [==============================] - 1s - loss: 0.3388 - acc: 0.8639     
Epoch 97/100
7200/7200 [==============================] - 1s - 

7200/7200 [==============================] - 0s - loss: 0.3930 - acc: 0.8365     
Epoch 73/100
7200/7200 [==============================] - 0s - loss: 0.3934 - acc: 0.8379     
Epoch 74/100
7200/7200 [==============================] - 0s - loss: 0.3938 - acc: 0.8360     
Epoch 75/100
7200/7200 [==============================] - 0s - loss: 0.3933 - acc: 0.8356     
Epoch 76/100
7200/7200 [==============================] - 0s - loss: 0.3927 - acc: 0.8378     
Epoch 77/100
7200/7200 [==============================] - 0s - loss: 0.3930 - acc: 0.8374     
Epoch 78/100
7200/7200 [==============================] - 0s - loss: 0.3933 - acc: 0.8371     
Epoch 79/100
7200/7200 [==============================] - 0s - loss: 0.3933 - acc: 0.8369     
Epoch 80/100
7200/7200 [==============================] - 0s - loss: 0.3930 - acc: 0.8378     
Epoch 81/100
7200/7200 [==============================] - 0s - loss: 0.3933 - acc: 0.8385     
Epoch 82/100
7200/7200 [==============================] - 0s - 

7200/7200 [==============================] - 1s - loss: 0.3939 - acc: 0.8371     
Epoch 58/100
7200/7200 [==============================] - 2s - loss: 0.3936 - acc: 0.8378     
Epoch 59/100
7200/7200 [==============================] - 1s - loss: 0.3937 - acc: 0.8365     
Epoch 60/100
7200/7200 [==============================] - 1s - loss: 0.3938 - acc: 0.8385     
Epoch 61/100
7200/7200 [==============================] - 1s - loss: 0.3938 - acc: 0.8358     
Epoch 62/100
7200/7200 [==============================] - 2s - loss: 0.3933 - acc: 0.8364     
Epoch 63/100
7200/7200 [==============================] - 1s - loss: 0.3934 - acc: 0.8364     
Epoch 64/100
7200/7200 [==============================] - 1s - loss: 0.3931 - acc: 0.8361     
Epoch 65/100
7200/7200 [==============================] - 2s - loss: 0.3934 - acc: 0.8371     
Epoch 66/100
7200/7200 [==============================] - 2s - loss: 0.3932 - acc: 0.8350     
Epoch 67/100
7200/7200 [==============================] - 1s - 

7200/7200 [==============================] - 0s - loss: 0.3985 - acc: 0.8365     
Epoch 43/100
7200/7200 [==============================] - 0s - loss: 0.3983 - acc: 0.8372     
Epoch 44/100
7200/7200 [==============================] - 0s - loss: 0.3982 - acc: 0.8368     
Epoch 45/100
7200/7200 [==============================] - 0s - loss: 0.3978 - acc: 0.8375     
Epoch 46/100
7200/7200 [==============================] - 0s - loss: 0.3981 - acc: 0.8379     
Epoch 47/100
7200/7200 [==============================] - 0s - loss: 0.3973 - acc: 0.8354     
Epoch 48/100
7200/7200 [==============================] - 0s - loss: 0.3976 - acc: 0.8368     
Epoch 49/100
7200/7200 [==============================] - 0s - loss: 0.3971 - acc: 0.8375     
Epoch 50/100
7200/7200 [==============================] - 0s - loss: 0.3969 - acc: 0.8350     
Epoch 51/100
7200/7200 [==============================] - 1s - loss: 0.3969 - acc: 0.8367     
Epoch 52/100
7200/7200 [==============================] - 0s - 

...

8000/8000 [==============================] - 1s - loss: 0.3990 - acc: 0.8373     
Epoch 154/500
8000/8000 [==============================] - 1s - loss: 0.3991 - acc: 0.8367     
Epoch 155/500
8000/8000 [==============================] - 1s - loss: 0.3989 - acc: 0.8346     
Epoch 156/500
8000/8000 [==============================] - 1s - loss: 0.3992 - acc: 0.8348     
Epoch 157/500
8000/8000 [==============================] - 1s - loss: 0.3991 - acc: 0.8363     
Epoch 158/500
8000/8000 [==============================] - 1s - loss: 0.3993 - acc: 0.8355     
Epoch 159/500
8000/8000 [==============================] - 1s - loss: 0.3993 - acc: 0.8356     
Epoch 160/500
8000/8000 [==============================] - 1s - loss: 0.3990 - acc: 0.8361     
Epoch 161/500
8000/8000 [==============================] - 1s - loss: 0.3987 - acc: 0.8359     
Epoch 162/500
8000/8000 [==============================] - 1s - loss: 0.3994 - acc: 0.8365     
Epoch 163/500
8000/8000 [=============================

8000/8000 [==============================] - 1s - loss: 0.3986 - acc: 0.8351     
Epoch 239/500
8000/8000 [==============================] - 1s - loss: 0.3988 - acc: 0.8360     
Epoch 240/500
8000/8000 [==============================] - 1s - loss: 0.3990 - acc: 0.8363     
Epoch 241/500
8000/8000 [==============================] - 1s - loss: 0.3989 - acc: 0.8363     
Epoch 242/500
8000/8000 [==============================] - 1s - loss: 0.3986 - acc: 0.8367     
Epoch 243/500
8000/8000 [==============================] - 1s - loss: 0.3985 - acc: 0.8370     
Epoch 244/500
8000/8000 [==============================] - 1s - loss: 0.3987 - acc: 0.8355     
Epoch 245/500
8000/8000 [==============================] - 1s - loss: 0.3987 - acc: 0.8364     
Epoch 246/500
8000/8000 [==============================] - 1s - loss: 0.3989 - acc: 0.8361     
Epoch 247/500
8000/8000 [==============================] - 1s - loss: 0.3985 - acc: 0.8360     
Epoch 248/500
8000/8000 [=============================

8000/8000 [==============================] - 1s - loss: 0.3987 - acc: 0.8364     
Epoch 324/500
8000/8000 [==============================] - 1s - loss: 0.3986 - acc: 0.8364     
Epoch 325/500
8000/8000 [==============================] - 1s - loss: 0.3986 - acc: 0.8373     
Epoch 326/500
8000/8000 [==============================] - 1s - loss: 0.3985 - acc: 0.8361     
Epoch 327/500
8000/8000 [==============================] - 1s - loss: 0.3986 - acc: 0.8360     
Epoch 328/500
8000/8000 [==============================] - 1s - loss: 0.3987 - acc: 0.8370     
Epoch 329/500
8000/8000 [==============================] - 1s - loss: 0.3987 - acc: 0.8360     
Epoch 330/500
8000/8000 [==============================] - 1s - loss: 0.3985 - acc: 0.8357     
Epoch 331/500
8000/8000 [==============================] - 1s - loss: 0.3986 - acc: 0.8355     
Epoch 332/500
8000/8000 [==============================] - 1s - loss: 0.3986 - acc: 0.8364     
Epoch 333/500
8000/8000 [=============================

8000/8000 [==============================] - 1s - loss: 0.3988 - acc: 0.8364     
Epoch 409/500
8000/8000 [==============================] - 1s - loss: 0.3983 - acc: 0.8356     
Epoch 410/500
8000/8000 [==============================] - 1s - loss: 0.3987 - acc: 0.8354     
Epoch 411/500
8000/8000 [==============================] - 1s - loss: 0.3984 - acc: 0.8356     
Epoch 412/500
8000/8000 [==============================] - 1s - loss: 0.3989 - acc: 0.8357     
Epoch 413/500
8000/8000 [==============================] - 1s - loss: 0.3986 - acc: 0.8357     
Epoch 414/500
8000/8000 [==============================] - 1s - loss: 0.3981 - acc: 0.8354     
Epoch 415/500
8000/8000 [==============================] - 1s - loss: 0.3983 - acc: 0.8359     
Epoch 416/500
8000/8000 [==============================] - 1s - loss: 0.3985 - acc: 0.8354     
Epoch 417/500
8000/8000 [==============================] - 1s - loss: 0.3985 - acc: 0.8361     
Epoch 418/500
8000/8000 [=============================

8000/8000 [==============================] - 1s - loss: 0.3983 - acc: 0.8355     
Epoch 494/500
8000/8000 [==============================] - 1s - loss: 0.3982 - acc: 0.8364     
Epoch 495/500
8000/8000 [==============================] - 1s - loss: 0.3987 - acc: 0.8369     
Epoch 496/500
8000/8000 [==============================] - 1s - loss: 0.3987 - acc: 0.8365     
Epoch 497/500
8000/8000 [==============================] - 1s - loss: 0.3984 - acc: 0.8365     
Epoch 498/500
8000/8000 [==============================] - 1s - loss: 0.3985 - acc: 0.8364     
Epoch 499/500
8000/8000 [==============================] - 1s - loss: 0.3986 - acc: 0.8354     
Epoch 500/500
8000/8000 [==============================] - 1s - loss: 0.3982 - acc: 0.8360     


In [44]:
best_parameters

{'batch_size': 32, 'epochs': 500, 'optimizer': 'rmsprop'}

In [45]:
best_accuracy

0.85799999999999998